In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

%reload_ext tensorboard

time: 311 ms (started: 2021-09-03 13:30:02 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #2 Sept 03, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main fb6a44a] 更新 #2 Sept 03, 2021
 2 files changed, 12 insertions(+)
 create mode 100644 "Lightning-Flash/.ipynb_checkpoints/\346\234\252\345\221\275\345\220\215-checkpoint.ipynb"
 create mode 100644 "Lightning-Flash/\346\234\252\345\221\275\345\220\215.ipynb"


To github.com:ustchope/pytorch_lightning_study.git
   46f24ee..fb6a44a  main -> main


time: 3.6 s (started: 2021-09-03 13:30:15 +08:00)


In [3]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from collections import OrderedDict

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything, Trainer, LightningDataModule
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy
import torchmetrics
import torchvision.transforms as transforms

seed_everything(7)

PATH_DATASETS = os.environ.get('PATH_DATASETS', '.')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

Global seed set to 7


time: 1.44 s (started: 2021-09-03 13:30:30 +08:00)


# 快速开始

Flash 是一个高级深度学习框架，用于快速原型设计、基线、微调和解决深度学习问题。 它具有一组任务供您用于开箱即用的推理和微调，以及一个易于实现的 API 来自定义流程的每个步骤以获得充分的灵活性。

Flash 是为初学者构建的，其简单的 API 需要很少的深度学习背景，也适合数据科学家、Kagglers、应用 ML 从业者和深度学习研究人员，他们希望通过 PyTorch Lightning 提供的高级功能快速获得深度学习基线。

# 为什么是Flash？

## 深度学习入门

### 简单易学

如果您刚刚开始使用深度学习，Flash 提供了常见的深度学习任务，您可以在几行代码中开箱即用，无需数学、花哨的 nn.Modules 或研究经验！

### 易于扩展

Flash 建立在 PyTorch Lightning 之上，PyTorch Lightning 是一个强大的深度学习研究框架，用于大规模训练模型。 借助 Lightning 的强大功能，您可以在任何硬件上训练您的 Flash 任务：CPU、GPU 或 TPU，无需更改任何代码。

### 易于提升技能

如果您想创建更复杂和自定义的模型，您可以使用 PyTorch 或 PyTorch Lightning 组件重构 flash 的任何部分，以获得您需要的所有灵活性。 闪电只是组织 PyTorch，抽象掉了不必要的工程细节。
* Flash（高级）
* Lightning（中级）
* PyTorch（低级）
当您需要更多灵活性时，您可以构建自己的任务或直接使用 Lightning。

## 用于深度学习研究

### 达到基线的最快方法

PyTorch Lightning 旨在抽象掉不必要的样板，同时实现最大的灵活性。为了提供充分的灵活性，解决闪电中的分类等非常常见的深度学习问题仍然需要一些样板。在新数据集或域外任务上运行基线模型仍然需要相当长的时间。我们创建 Flash 是为了满足我们的用户对使用经过验证的常用数据模式主干为 Lightning 建立基线的超快速方法的需求。 Flash 旨在成为您研究的最简单起点 - 从 Flash 任务开始作为基准，并在您进行 SOTA 研究的途中使用 Lightning 或 PyTorch 组件覆盖 Flash 的任何部分。

### 随心所欲的灵活性

Flash 任务本质上是 LightningModules，Flash Trainer 是 Lightning Trainer 的薄包装。您可以使用自己的 LightningModule 代替 Flash 任务，使用 Lightning Trainer 代替 Flash 训练器等。Flash 帮助您更加专注于您的研究，而不是其他任何事情。

### 标准最佳实践

Flash 任务为各种不同的模型和域实施标准的最佳实践，以节省您挖掘不同实施的时间。 Flash 比 Lightning 抽象出更多的细节，允许深度学习专家分享他们解决范围深度学习问题的技巧和窍门。

# 任务

Flash 由一组任务组成。 Flash 任务是激光聚焦的对象，旨在使用最先进的方法解决明确定义的问题类型。

Flash 任务包含解决手头任务的所有相关信息——您要预测的类标签数量、数据集中的列数，以及所用模型架构的详细信息，例如损失函数、优化器等。

以下是任务示例：

In [ ]:
from flash.text import TextClassifier
from flash.image import ImageClassifier
from flash.tabular import TabularClassifier

> 根据定义，任务是不灵活的！ 为了获得更大的灵活性，您可以直接使用 LightningModule 或只需几行即可修改现有任务。

# 推理

推理是从经过训练的模型生成预测的过程。 使用任务进行推理：
1. 使用检查点（检查点只是捕获模型使用的所有参数的确切值的文件）使用预训练权重初始化您的任务。 本地文件或 URL 有效。
2. 将数据传递给 flash.core.model.Task.predict()。

下面是一个推理的例子：

In [ ]:
# import our libraries
from flash.text import TextClassifier

# 1. Init the finetuned task from URL
model = TextClassifier.load_from_checkpoint("https://flash-weights.s3.amazonaws.com/text_classification_model.pt")

# 2. Perform inference from list of sequences
predictions = model.predict(
    [
        "Turgid dialogue, feeble characterization - Harvey Keitel a judge?.",
        "The worst movie in the history of cinema.",
        "This guy has done a great job with this movie!",
    ]
)
print(predictions)

我们得到以下输出：

In [ ]:
["negative", "negative", "positive"]

# 微调

微调（或迁移学习）是将在大型数据集上训练的模型调整为您的特定（可能小得多）数据集的过程。 所有 Flash 任务都有预训练的主干，这些主干已经在 ImageNet 等大型数据集上进行了训练。 对预训练模型进行微调可显着减少训练时间。

要使用任务进行微调：
* 使用为任务定制的 DataModule 加载您的数据并组织它（例如：ImageClassificationData）。
* 选择并初始化您的任务，其中内置了最先进的主干（例如：ImageClassifier）。
* 初始化 flash.core.trainer.Trainer。
* 选择微调策略（例如：“冻结”）并使用您的数据调用 flash.core.trainer.Trainer.finetune()。
* 保存您的微调模型。

这是一个微调的例子。

In [ ]:
from pytorch_lightning import seed_everything

import flash
from flash.core.classification import Labels
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier

# set the random seeds.
seed_everything(42)

# 1. Download and organize the data
download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", "data/")

datamodule = ImageClassificationData.from_folders(
    train_folder="data/hymenoptera_data/train/",
    val_folder="data/hymenoptera_data/val/",
    test_folder="data/hymenoptera_data/test/",
)

# 2. 使用所需的任务构建模型
model = ImageClassifier(backbone="resnet18", num_classes=datamodule.num_classes)

# 3. 创建训练器（运行一个 epoch 进行演示）
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())

# 4. 微调模型
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

# 5. 保存模型！
trainer.save_checkpoint("image_classification_model.pt")

## 使用微调模型

微调后，使用模型进行预测：

In [ ]:
# 将预测序列化为标签，从第 2 部分中的训练数据自动推断。
model.serializer = Labels()

predictions = model.predict(
    [
        "data/hymenoptera_data/val/bees/65038344_52a45d090d.jpg",
        "data/hymenoptera_data/val/ants/2255445811_dabcdf7258.jpg",
    ]
)
print(predictions)

我们得到以下输出：

In [ ]:
['bees', 'ants']

或者您可以在任何地方使用保存的模型进行预测！

In [ ]:
from flash.image import ImageClassifier

# load finetuned checkpoint
model = ImageClassifier.load_from_checkpoint("image_classification_model.pt")

predictions = model.predict("path/to/your/own/image.png")

# 训练

当您有足够的数据时，最好从头开始训练而不是微调。

要从头开始训练任务：
1. 使用为任务定制的 DataModule 加载您的数据并组织它（例如：ImageClassificationData）。
2. 选择并初始化您的任务（设置 pretrained=False），它内置了最先进的主干（例如：ImageClassifier）。
3. 初始化 flash.core.trainer.Trainer 或 pytorch_lightning.trainer.Trainer。
4. 使用您的数据集调用 flash.core.trainer.Trainer.fit()。
5. 保存经过训练的模型。

下面是一个例子：

In [ ]:
from pytorch_lightning import seed_everything

import flash
from flash.core.classification import Labels
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier

# set the random seeds.
seed_everything(42)

# 1. Download and organize the data
download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", "data/")

datamodule = ImageClassificationData.from_folders(
    train_folder="data/hymenoptera_data/train/",
    val_folder="data/hymenoptera_data/val/",
    test_folder="data/hymenoptera_data/test/",
)

# 2. Build the model using desired Task
model = ImageClassifier(backbone="resnet18", num_classes=datamodule.num_classes, pretrained=False)

# 3. Create the trainer (run one epoch for demo)
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())

# 4. Train the model
trainer.fit(model, datamodule=datamodule)

# 5. Save the model!
trainer.save_checkpoint("image_classification_model.pt")

# 一些内置任务
* 图像分类
* 图像嵌入器
* 文本分类
* 总结任务
* 翻译任务
* 表格分类

更多任务即将推出！

In [8]:
pip install 'lightning-flash[all]' -U

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/8f/d3/d994f9347b42407adc04e58fdeb5e52013df14bcc4a678c5211ffd526ebd/pandas-1.2.5-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (9.7 MB)
     |████████████████████████████████| 376 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 281 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 159 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 37.7 MB 184 kB/s eta 0:00:01
     |████████████████████████████████| 23.7 MB 2.3 MB/s eta 

In [10]:
pip install plotly -U

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 23.9 MB 2.8 MB/s eta 0:00:01
  Attempting uninstall: plotly
    Found existing installation: plotly 4.14.3
    Uninstalling plotly-4.14.3:
      Successfully uninstalled plotly-4.14.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fiftyone 0.13.1 requires plotly<5,>=4.14, but you have plotly 5.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
time: 21.4 s (started: 2021-09-03 13:53:14 +08:00)
